In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService


import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import sys
import pprint
max_chrome_driver_path = '/Users/maxlien/Downloads/chromedriver-mac-arm64/chromedriver'

from dotenv import load_dotenv
import os

def config():
    load_dotenv()
  
config()

username = str(os.getenv('MONOPLE_GMAIL'))
password=str(os.getenv('MONOPLE_PASSWORD'))
service = ChromeService(executable_path=max_chrome_driver_path)
service)

In [26]:
class StackExchangeScraper:
    def __init__(self, chrome_driver_path=max_chrome_driver_path,username=username, password=password):
        self.driver_path = chrome_driver_path
        self.options = webdriver.ChromeOptions()
        #print(self.options)
        self.service = ChromeService(executable_path=self.driver_path)
        self.driver = webdriver.Chrome(service=self.service) #executable_path= max_chrome_driver_path)#,options=self.options)#executable_path=self.driver_path)
        self.wait = WebDriverWait(self.driver, 10)
        
        self.username =username
        self.password = password

    # ... rest of the class methods remain unchanged ...

# Note: Only the initialization method of the class is shown here for brevity.
# The rest of the class methods would remain unchanged from the previous version.
    def _set_up_agent(self):
        #set useragent
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    # def _setup_headless_driver(self):
    #     """
    #     Sets up a headless Selenium Chrome driver.
    #     """
    #     self.options.add_argument('--headless')
    #     self.options.add_argument('--no-sandbox')
    #     self.options.add_argument('--disable-dev-shm-usage')
    #     self.driver = webdriver.Chrome(self.driver_path, options=options)
    def _wait(self,type, name):
        return self.wait.until(
                EC.presence_of_element_located((type, name))
            )
    def _wait_all(self, type, name):
        return self.wait.until(
                EC.visibility_of_all_elements_located((type, name))
            )
    def _login(self, username, password):
        
        
        login_button_XPATH = '/html/body/div[1]/header[1]/div/nav/ol/li[4]/a'
        login_button = self._wait(By.XPATH, login_button_XPATH)
        self.driver.execute_script("arguments[0].scrollIntoView();", login_button)
        login_button.click()


        
    

        username_box = self._wait(By.XPATH, '//*[@id="email"]')
        username_box.send_keys(username)

        password_box = self._wait(By.XPATH,'//*[@id="password"]' )
        password_box.send_keys(password)


        submit_button =self._wait(By.XPATH, '//*[@id="submit-button"]') 
        self.driver.execute_script("arguments[0].scrollIntoView();", submit_button)
        submit_button.click()




    def search_links(self, input):
        self._set_up_agent()
        self.driver.get('https://stackexchange.com')

        self._login(self.username, self.password)

        search_box = self._wait(By.XPATH, '//*[@id="search"]/div/input')
        search_box.send_keys(input)
        search_box.send_keys(Keys.ENTER)


        content = self._wait(By.XPATH, '//*[@id="content"]/div/div[3]')
        soup = BeautifulSoup(content, parser='html.parser')
        links = soup.find_all(class_='result-link')
        # for link in links:
        #     print(link.a['href'])
        urls = [link.a['href'] for link in links]
        return urls       
        # try:
        #     #for stack_exchange
        #     #result_link_elements = self._wait_all(By.CSS_SELECTOR, "result-link > a")
        #     #print(result_link_elements)
        #     # Extracting the links within each of the 'result-link' elements
        #     links =[]
        #     for element in self._wait_all(By.CSS_SELECTOR, "result-link > a"):
        #         links.append(element.get_attribute("herf"))
        # except TimeoutException :
        #     #in case it redirect to meta.stackexchange
        #     #result_link_elements = self._wait_all(By.CLASS_NAME, "s-post-summary--content-title > a")
        #     #print(result_link_elements)
        #     # Extracting the links within each of the 'result-link' elements
        #     links = []
        #     for element in self._wait_all(By.CSS_SELECTOR, "s-post-summary--content-title > a"):
        #         links.append(element.get_attribute("innerHTML"))

        
        # return links


    def search_anwser(self,url):
        answer = dict()
        title_XPATH = '//*[@id="question-header"]/h1/a'
        answer_body_CLASS = 's-prose js-post-body'
        self.driver.get(url)
        title = self._wait(By.XPATH, title_XPATH)

        answer_body = self._wait(By.CLASS_NAME, answer_body_CLASS)

        answer['title'] = title.text
        answer['answer'] = answer_body.text

        return answer
        



    def scrape(self, input):
        """
        Main method to scrape StackExchange for a given query and return the texts in a list.
        """
        #self._setup_headless_driver()
        
        urls = self.search_links(input)
        for url in urls:
            self.search_anwser(url)

        self.driver.quit()

        return elements

# Note: As before, the Selenium portion cannot be executed in this environment. 
# The code should be run in a local environment with the appropriate browser driver.


In [27]:
SS = StackExchangeScraper(chrome_driver_path=max_chrome_driver_path,username=username, password=password)
content =SS.scrape('what is positive matter')
content

'\n                <div class="question search-result" id="question-summary-55769540">\n    <div class="hot-question-site-icon">\n        <a href="https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what" title="view this question on Stack Overflow"><img src="https://cdn.sstatic.net/Sites/stackoverflow/Img/icon-48.png"></a>\n    </div>\n    <div class="statscontainer">\n        <div class="stats">\n            <div class="vote">\n                <div class="votes">\n                    <span class="vote-count-post "><strong>0</strong></span>\n                    <div class="viewcount">votes</div>\n                </div>\n            </div>\n                <div class="status answered">\n                    <strong>2</strong>answers\n                </div>\n        </div>\n    </div>\n    <div class="summary">\n        <div class="result-link">\n            <span><a href="https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-wh

In [50]:
soup = BeautifulSoup(content, parser='html.parser')
links = soup.find_all(class_='result-link')
for link in links:
    print(link.a['href'])
# links = [link.a['href'] for link in a]
# print(len(links))


https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what
https://math.stackexchange.com/questions/3665900/determining-whether-a-matrix-is-positive-definite-from-its-lu-decomposition
https://electronics.stackexchange.com/questions/499491/splicing-ac-adaptor-cord-which-is-positive-and-which-is-negative
https://electronics.stackexchange.com/questions/227190/why-doesnt-positive-negative-terminal-matter-in-some-electrical-applicances
https://stackoverflow.com/questions/7129597/how-to-keep-track-of-a-position-no-matter-what-screen-size-is
https://stackoverflow.com/questions/74447652/if-the-value-in-sql-is-positive-how-can-i-do-this-operation-if-it-is-negative
https://stackoverflow.com/questions/69641323/why-codeblocks-asking-me-to-add-a-after-elses-condition-if-i-add-a-th
https://worldbuilding.stackexchange.com/questions/101581/negative-energy-beam-negative-matter-projectiles
https://mathematica.stackexchange.com/questions/28659/manipulate-doesnt-work-for-plott

In [ ]:
driver.get('https://stackoverflow.com/questions/55769540/position-sticky-not-working-no-matter-what')


In [120]:

soup = BeautifulSoup(content, parser='html.parser')
#links = content.text
links = soup.find_all('a')
herfs = []
for link in links:
    herf = link.get('herf')
    herfs.append(herf)

TypeError: object of type 'WebElement' has no len()

In [ ]:
herfs

In [71]:
soup = BeautifulSoup(parser='html.parser', content=result)
soup.find_all('herf')

TypeError: __init__() got an unexpected keyword argument 'content'

In [67]:
dir(result[0])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']